In [2]:
print("start")

start


In [4]:
import pandas as pd
import re

df = pd.read_excel (r'vicinitas_search_results.xlsx') #dataframe from ecxel
# your own datafram from twitter api output

print (df)
tweets = df.to_numpy()
#print (tweets)

#remove urls from tweets 
no_urls=[]
for i in tweets :
    text = re.sub(r'http\S+', '', str(i))
    no_urls.append(text)
#print(no_urls)

#remove emojis from text
def deEmojify(text):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return regrex_pattern.sub(r'',text)

no_emoji = []
for i in no_urls :
    text = deEmojify(i)
    no_emoji.append(text)
#print(no_emoji)

#remove mentions 

no_mention = [ re.sub(r"(?:\@|https?\://)\S+", "", i) for i in no_emoji ]
#print(no_mention[16])

# remove punctuation

def remove_punctuation(words):
    
    new_words = []
    
 
    for word in words:
        new_word = re.sub(r'[^\w\s\,]', '', (word))
        if new_word != '':
               new_words.append(new_word)
    return new_words

no_punc = [re.sub(r'[^\w\s]', '',i) for i in no_mention]
#print(no_punc)

no_commas = [re.sub('RT', '', i) for i in no_punc]
no_commas = [re.sub(',', '', i) for i in no_commas]
no_commas = [re.sub('#', '', i) for i in no_commas]
no_commas = [re.sub('n', ' ', i) for i in no_commas]
no_commas = [re.sub("n", ' ', i) for i in no_commas]
no_commas = [re.sub("_", ' ', i) for i in no_commas]





#print(no_commas)

#pd.DataFrame(no_commas).to_excel('output.xlsx', header=False, index=False)


#tokenization 

import nltk
#nltk.download('punkt')

nltk_tokens = nltk.word_tokenize(no_commas[17])
print (nltk_tokens[0])

#filter weather tweets from non weather tweets :

def filtertweets(tweets) : 
    
    weathertweets = []
    for tweet in tweets :
        
        tweet_tokens = nltk.word_tokenize(tweet)
        if "الطقس" in tweet_tokens : 
            weathertweets.append(tweet)
            #print(tweet, end = " ***")
    
    
    return weathertweets

weather_tweets = filtertweets(no_commas)
#print(weather_tweets)


#stemming arabic words
from nltk.stem.isri import ISRIStemmer
stemmer = ISRIStemmer()

#print(weather_tweets[0])
stemmed_tweets = []
for i in weather_tweets:
    tweet_tokens = nltk.word_tokenize(i)
    #print(tweet_tokens)
    stem_tweet = []
    for j in tweet_tokens : 
        st = stemmer.stem(j)
        stem_tweet.append(st)
        
    stemmed_tweets.append(stem_tweet)
#print(stemmed_tweets)

# keywords counting :
keywords = ['برد', 'مطر', 'شمس']
countlist = [0,0,0]


for i in stemmed_tweets : 
    for j in keywords :
        count = i.count(j)
        index = keywords.index(j)
        countlist[index]+= count
        #print(count)
    
print(keywords)
print(countlist)




    
    

    
#print(i)
#print(st)


                                                   Text
0     طقس مائل للبرودة والرياح متقلبة الاتجاه\n\n#ال...
1     #إنفوجرافيك_الراية| حالة #الطقس في دولة #قطر ل...
2     @MeteoAlgerie_  توقعات #الطقس لولاية #إليزي ht...
3     RT @MekshatWorld : 📽️ #شاهد \nكهف فينغال البحر...
4     RT @MekshatWorld : 📽️ #شاهد \nجريان وادي الحلو...
...                                                 ...
1405  RT @emara_hail : #تاريخ_اليوم \n#مواقيت_الصلاة...
1406  ▪️  #الطقس : انخفاض في الحرارة تشهده البلاد وف...
1407  حالة البحر اليوم ..\n\n#الكويت \n#الطقس \n#طقس...
1408  RT @bawabaanews : عاصفة متوسطية تتجه خلال السا...
1409  RT @SPAregions : #الطقس: بمشيئة الله تعالى؛ تت...

[1410 rows x 1 columns]
حالة
['برد', 'مطر', 'شمس']
[91, 376, 19]


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from keras.models import load_model


model = Sequential()

model.x = Input(shape=(3,)) # change 3 by numer of items / keywords
model.add(Dense(12,  activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(3, activation = 'softmax')) #change 3 by numer of items / keywords

tr = pd.read_excel (r'trainings.xlsx') #dataframe from ecxel
xtrain = tr.iloc[:90,:3].to_numpy()  # [lign begin :lign end ,column begin : column end] 
xlabels = tr.iloc[:90,3:6].to_numpy()
xval = tr.iloc[90:111,:3].to_numpy()
xvalabels = tr.iloc[90:111,3:6].to_numpy()

print(xtrain.shape)
print(xlabels.shape)
print(xval.shape)
print(xvalabels.shape)



model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(         #function to run training
    xtrain,
    xlabels,
    batch_size=5,   #number of data we train at the same time
    epochs=200,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(xval, xvalabels),
)

#print(xlabels)
#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

pred = model.predict([list(countlist)]) 
pre_sorted = sorted(pred[0], reverse = True)
print(pre_sorted)
print(pred[0])

index1 = list(pred[0]).index(pre_sorted[0])
index2 = list(pred[0]).index(pre_sorted[1])

value1 = keywords[index1]
value2 = keywords[index2]

print(value1)
print(value2)




print('the weather is {}, and a bit {} in arabia saoudia'.format(value1,value2) ) 







(90, 3)
(90, 3)
(21, 3)
(21, 3)
Epoch 1/200
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 1/18 [>.............................] - ETA: 6s - loss: 1.6560 - accuracy: 0.0000e+00WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000029F387EB558> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Lin

Epoch 45/200
18/18 [==============================] - 0s 2ms/step - loss: 1.0074 - accuracy: 0.5667 - val_loss: 1.0388 - val_accuracy: 0.7619
Epoch 46/200
18/18 [==============================] - 0s 2ms/step - loss: 1.0078 - accuracy: 0.6000 - val_loss: 1.0414 - val_accuracy: 0.7143
Epoch 47/200
18/18 [==============================] - 0s 2ms/step - loss: 1.0078 - accuracy: 0.6000 - val_loss: 1.0362 - val_accuracy: 0.6667
Epoch 48/200
18/18 [==============================] - 0s 2ms/step - loss: 1.0070 - accuracy: 0.5889 - val_loss: 1.0393 - val_accuracy: 0.7619
Epoch 49/200
18/18 [==============================] - 0s 2ms/step - loss: 1.0057 - accuracy: 0.5889 - val_loss: 1.0365 - val_accuracy: 0.7619
Epoch 50/200
18/18 [==============================] - 0s 3ms/step - loss: 1.0068 - accuracy: 0.6111 - val_loss: 1.0319 - val_accuracy: 0.8095
Epoch 51/200
18/18 [==============================] - 0s 3ms/step - loss: 1.0087 - accuracy: 0.5889 - val_loss: 1.0275 - val_accuracy: 0.7619
Epoch 

Epoch 103/200
18/18 [==============================] - 0s 3ms/step - loss: 0.9833 - accuracy: 0.6556 - val_loss: 1.0184 - val_accuracy: 0.7619
Epoch 104/200
18/18 [==============================] - 0s 3ms/step - loss: 0.9829 - accuracy: 0.6111 - val_loss: 1.0150 - val_accuracy: 0.7619
Epoch 105/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9841 - accuracy: 0.6333 - val_loss: 1.0207 - val_accuracy: 0.7619
Epoch 106/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9851 - accuracy: 0.6111 - val_loss: 1.0182 - val_accuracy: 0.7143
Epoch 107/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9819 - accuracy: 0.6222 - val_loss: 1.0157 - val_accuracy: 0.7143
Epoch 108/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9809 - accuracy: 0.6333 - val_loss: 1.0164 - val_accuracy: 0.7143
Epoch 109/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9801 - accuracy: 0.6222 - val_loss: 1.0143 - val_accuracy: 0.7619

18/18 [==============================] - 0s 2ms/step - loss: 0.9760 - accuracy: 0.6667 - val_loss: 1.0023 - val_accuracy: 0.8095
Epoch 161/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9777 - accuracy: 0.6556 - val_loss: 1.0028 - val_accuracy: 0.8095
Epoch 162/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9755 - accuracy: 0.6667 - val_loss: 1.0034 - val_accuracy: 0.7619
Epoch 163/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9741 - accuracy: 0.6556 - val_loss: 1.0009 - val_accuracy: 0.8571
Epoch 164/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9743 - accuracy: 0.6889 - val_loss: 1.0046 - val_accuracy: 0.7619
Epoch 165/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9783 - accuracy: 0.6778 - val_loss: 1.0008 - val_accuracy: 0.7143
Epoch 166/200
18/18 [==============================] - 0s 2ms/step - loss: 0.9755 - accuracy: 0.6444 - val_loss: 1.0014 - val_accuracy: 0.8571
Epoch 167/200